In [ ]:
import numpy as np
#import sympy as sp
import matplotlib.pyplot as plt


#DEFINING CONSTANTS:

L_1 = 10
L_2 = 26
L_3  = 18
G = 20
omega = 1.5 #rad/s


#Points of joints
s_1_p1 = np.array([[-L_1/2], [0]])
s_1_p2 = np.array([[L_1/2], [0]])
s_2_p2 = np.array([[-L_2/2], [0]])
s_2_p3 = np.array([[L_2/2], [0]])
s_3_p3 = np.array([[-L_3/2], [0]])
s_3_p4 = np.array([[L_3/2], [0]])

#Constraint distances
c_1 = np.zeros((2,1))
c_2 = c_1
c_3 = c_2
c_4 = np.array([[20], [0]])

""" # r-vectors
r_1 = np.array([[x_1], [y_1]])
r_2 = np.array([[x_2], [y_2]])
r_3 = np.array([[x_3], [y_3]])
 """

#A-matrix
def A_matrix(phi):
    A_matrix = np.squeeze(np.array([[np.cos(phi), -np.sin(phi)], [np.sin(phi), np.cos(phi)]]))
    return A_matrix

def B_matrix(phi):
    B_matrix = np.squeeze(np.array([[-np.sin(phi), -np.cos(phi)], [np.cos(phi), -np.sin(phi)]]))
    return B_matrix

#Phi equations
def phi_rel(r_mat, phi_vec, s_p_mat, c_vec):
    r_vec_1 = r_mat[:,0].reshape(2,1)
    r_vec_2 = r_mat[:,1].reshape(2,1)
    s_p_mat_1 = s_p_mat[:,0].reshape(2,1)
    s_p_mat_2 = s_p_mat[:,1].reshape(2,1)
    c_vec = c_vec.reshape(2,1)
    relative_constraint_result = (r_vec_1 + A_matrix(phi_vec[0]) @ s_p_mat_1) - (r_vec_2 + A_matrix(phi_vec[1]) @ s_p_mat_2) - c_vec
    return relative_constraint_result

def phi_abs(r_vec, phi_elem, s_p_vec, c_vec):
    r_vec_1 = r_vec.reshape(2,1)
    s_p_vec_1 = s_p_vec.reshape(2,1)
    c_vec = c_vec.reshape(2,1)
    absolute_constraint_result = (r_vec_1 + A_matrix(phi_elem[0]) @ s_p_vec_1) - c_vec
    return absolute_constraint_result

def phi_driver(phi_val, t, omega=omega):
    driving_constraint = phi_val - omega * t
    return driving_constraint

def phi_equations_system(q, t, verbose=False):
    r_1 = q[0:2].reshape(2,1)
    phi_1 = q[2:3].reshape(1,1)
    r_2 = q[3:5].reshape(2,1)
    phi_2 = q[5:6].reshape(1,1)
    r_3 = q[6:8].reshape(2,1)
    phi_3 = q[8:9].reshape(1,1)

    r_12= np.hstack((r_1, r_2))
    phi_12= np.vstack((phi_1, phi_2))
    r_23= np.hstack((r_2, r_3))
    phi_23= np.vstack((phi_2, phi_3))
    
    phi_abs_1 = phi_abs(r_vec=r_1, phi_elem=phi_1, s_p_vec=combined_s_p_vec[:,0], c_vec=combined_c_vec[:,0])
    phi_rel_1 = phi_rel(r_mat=r_12, phi_vec=phi_12, s_p_mat=combined_s_p_vec[:,1:3], c_vec=combined_c_vec[:,1])
    phi_rel_2 = phi_rel(r_mat=r_23, phi_vec=phi_23, s_p_mat=combined_s_p_vec[:,3:5], c_vec=combined_c_vec[:,2])
    phi_abs_2 = phi_abs(r_vec=r_3, phi_elem=phi_3, s_p_vec=combined_s_p_vec[:,5], c_vec=combined_c_vec[:,3])
    phi_ad_1 = phi_driver(phi_1, t, omega=omega)

    vector_of_equations = np.vstack(phi_abs_1, phi_rel_1, phi_rel_2, phi_abs_2, phi_ad_1)

    if verbose == True:
        print(f"phi abs 1 {phi_abs_1}")
        print(f"phi rel 1 {phi_rel_1}")
        print(f"phi rel 2 {phi_rel_2}")
        print(f"phi abs 2 {phi_abs_2}")

    return vector_of_equations

test_vector = np.array([[0, 5, np.pi/2, 0, 0, 0, 20, 9, np.pi/2]]).reshape(9,1)
combined_s_p_vec = np.hstack([s_1_p1, s_1_p2, s_2_p2, s_2_p3, s_3_p3, s_3_p4])
combined_c_vec = np.hstack([c_1, c_2, c_3, c_4])
#TEST CASE _|
test_r_vec = np.array([[1, 2], [0, 1]])
test_phi_vec = np.array([[0], [np.pi/2]])
test_s_p_vec = np.array([[1, -1], [0, 0]])
test_c_vec = np.array([[2], [0]])
#test = phi_rel(test_r_vec, test_phi_vec, test_s_p_vec, test_c_vec)
test = phi_abs(test_r_vec[:,0], test_phi_vec[0], test_s_p_vec[:,0], test_c_vec)
test = phi_equations_system(test_vector, 0)
#print(f"test: {test}")

def solve_systems():
    pass

def phi_jacobian(q):
    
    jacobian_matrix = np.array([[np.ones((2,2)), B_matrix(q[2]), np.zeros((2, 6))
    ]])

def newton_rhapson(function, initial_guess, jacobian, rtol=1e-3, max_iter=50, verbose=False):
    old_val = initial_guess
    iter = 0
    for iter in range(max_iter):

        if rtol < norm:
            
            function_value = function(old_val, t)
            jacobian_value_inv = np.linalg.inv(jacobian(old_val))

            current_val = old_val - function_value @ jacobian_value_inv
            norm = np.linalg.norm(old_val - current_val)
            old_val = current_val
        else:
            break

        iter+= 1

    if verbose == True:
        print(f"The Newton Rhapson finished in {iter} iterations with a tolerance of {norm}")
    return current_val
""" 
def system_equations(q, t, s_p):
    system_equations = np.array([
        [q[0]-],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    ])"""

[[ 6.123234e-17 -1.000000e+00]
 [ 1.000000e+00  6.123234e-17]]
